In [3]:
%%capture
# update or install the necessary libraries
!pip install --upgrade google-search-results


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.agents import Tool, load_tools
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from virtual_assistant.tools import BrowsingTool
from virtual_assistant.utilities import GoogleSerperAPIWrapper

In [3]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:
# tool_names = ['google-serper']
# tools = load_tools(tool_names, llm=llm)

tools = [BrowsingTool(serper=GoogleSerperAPIWrapper())]

In [5]:
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

In [6]:
agent_chain.run("what are some good dinners to make this week, if i like thai food?")



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "list of thai food dinners to make this week"
}

AttributeError: 'NoneType' object has no attribute 'as_retriever'

In [7]:
agent_chain.run(input="tell me who won the world cup in 2022?")



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "Who won the world cup in 2022?"
}
Observation: 2022 World Cup / Champion :  Argentina national football team
Thought:{
    "action": "Final Answer",
    "action_input": "The winner of the 2022 World Cup is Argentina national football team."
}

> Finished chain.


'The winner of the 2022 World Cup is Argentina national football team.'

In [11]:
agent_chain.run('Oscar best movie in 2023.')



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "What was the Oscar best movie in 2023"
}
Observation: Academy Award for Best Picture / Winners (2023) :  Everything Everywhere All at Once
Thought:{
    "action": "Final Answer",
    "action_input": "The Academy Award for Best Picture winner in 2023 was 'Everything Everywhere All at Once'."
}

> Finished chain.


"The Academy Award for Best Picture winner in 2023 was 'Everything Everywhere All at Once'."